In [1]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np
import pickle
import os
from llama_cpp import Llama
import glob
    

In [2]:

def load_pdf_chunks(folder_path):
    chunks = []
    sources = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    for pdf_file in glob.glob(f"{folder_path}/*.pdf"):
        reader = PdfReader(pdf_file)
        text = "".join(page.extract_text() or "" for page in reader.pages)
        pdf_chunks = splitter.split_text(text)
        chunks.extend(pdf_chunks)
        sources.extend([os.path.basename(pdf_file)] * len(pdf_chunks))
    
    return chunks, sources

chunks, sources = load_pdf_chunks("data")

print(f"Se generaron {len(chunks)} chunks de {len(set(sources))} PDFs.")

Se generaron 29 chunks de 2 PDFs.


In [3]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
vectors = embedding_model.encode(chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

# Guardar index y chunks
os.makedirs("index", exist_ok=True)
with open("index/chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)
with open("index/sources.pkl", "wb") as f:
    pickle.dump(sources, f)
faiss.write_index(index, "index/faiss.index")
print("Index guardado.")

Index guardado.


In [4]:
llm = Llama(model_path="models/llama-2-7b-chat.Q2_K.gguf", n_ctx=2048)
print("LLM cargado correctamente.")    

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32           

LLM cargado correctamente.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'general.name': 'LLaMA v2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'llama.rope.dimension_count': '128', 'llama.embedding_length': '4096', 'llama.block_count': '32', 'llama.feed_forward_length': '11008', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '2', 'general.file_type': '10', 'llama.attention.head_count_kv': '32', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'tokenizer.ggml.model': 'llama', 'general.quantization_version': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.unknown_token_id': '0'}
Using fallback chat format: llama-2


In [5]:
def retrieve_context(query, k=3):
    q_vec = embedding_model.encode([query], convert_to_numpy=True)
    D, I = index.search(q_vec, k)
    return [(chunks[i], sources[i]) for i in I[0]]

def generate_answer(query):
    results = retrieve_context(query)

    # Obtener fuentes únicas utilizadas
    archivos_utilizados = set([src for _, src in results])

    # Construir contexto
    context = "\n".join([f"[{src}]: {chunk}" for chunk, src in results])

    # Prompt LLM
    prompt = f"Contexto:\n{context}\n\nPregunta: {query}\nRespuesta:"
    output = llm(prompt, max_tokens=300)
    respuesta = output["choices"][0]["text"]

    # Mostrar archivos utilizados junto a la respuesta
    print("📄 Archivos utilizados:")
    for archivo in archivos_utilizados:
        print(f" - {archivo}")

    return respuesta.strip()


# Probar pregunta
generate_answer("¿Que es ser un cliente politicamente expuesto?")
    

llama_perf_context_print:        load time =   61678.50 ms
llama_perf_context_print: prompt eval time =   61675.18 ms /   576 tokens (  107.07 ms per token,     9.34 tokens per second)
llama_perf_context_print:        eval time =   66758.29 ms /   299 runs   (  223.27 ms per token,     4.48 tokens per second)
llama_perf_context_print:       total time =  128738.22 ms /   875 tokens


📄 Archivos utilizados:
 - Personas Expuestas Políticamente (PEP).pdf


'Ser un cliente politicamente expuesto (PEP) significa que una persona ha desempeñado o desempeña funciones públicas destaca en un país, incluyendo a jefes de Estado o de un Gobierno, políticos de alta jerarquía, funcionarios gubernamentales, judiciales o militares de alta jerarquía, altos ejecutivos de empresas estatales, así como sus cónyuges, convivientes civiles y parientes hasta el segundo grado de consanguinidad (abuelos, nietos y hermanos), y las personas naturales con las que hayan tratado.\nLa Circular UAF N°62 define como Personas Expuestas Políticamente (PEP) a “los chilenos y extranjeros que desempeñen o hayan desempeñado funciones públicas destaca en un país, hasta a lo menos un año de finalizado el ejercicio de estas”.\nPara los clientes que se encuentran en el país, se recomienda verificar con el Ministerio de la Función Pública o con el servicio de impuestos local para obtener información detallada sobre las funciones públicas y las personas que deben ser consideradas c